In [2]:
%pip install -i https://pypi.gurobi.com gurobipy;
import gurobipy
import numpy as np
import pandas as pd

Looking in indexes: https://pypi.gurobi.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 53.2 MB/s eta 0:00:00


In [ ]:
C = []  # Set of classes
R = []  # Set of rooms (first 3 rooms are ORIE and 4th is extra)
T = []  # Set of time periods
L = []  # graph of nodes with edges between pairs (i,j) of classes that are forbidden to overlap​
F = []  # nodes with edges between pairs (n,m) of meeting patterns that overlap with each other​

# Courses

In [56]:
C_df = pd.DataFrame(pd.read_csv('new_courses_temp.csv',header=None))
C = C_df.sort_values(by=0).to_numpy()
C

array([[3120],
       [3150],
       [3310],
       [3510]])

# Overlap List

In [75]:
course_overlap_df = pd.DataFrame(pd.read_csv('course_overlap_temp.csv'))
course_overlap = course_overlap_df.to_numpy()

# check that all of the courses in the overlap list are in the course list
def is_subset(small_arr, big_arr):
    result = np.isin(small_arr, big_arr)
    return np.all(result)

assert is_subset(L, C), "Make sure C includes all values in L"

In [76]:
# Example of classes that cant overlap (i,l)
def overlap(n, forbidden_pairs):
    matrix = [[0] * n for _ in range(n)]

    for i, l in forbidden_pairs:
        if i <= n and l <= n:
            matrix[l][i] = 1
            matrix[i][l] = 1

    return matrix

In [77]:
def class_to_index(overlap_list, courses):
  # must do this becuase 0 is an index in the courses list and represents ORIE 3120
  index_matrix = np.zeros_like(overlap_list)

  for i, row in enumerate(overlap_list):
    for j, course in enumerate(row):
      index_matrix[i, j] = np.where(courses == course)[0][0]
  return index_matrix


In [79]:
index_course_overlap = class_to_index(course_overlap, C)
# print(index_course_overlap)
L = overlap(len(C), index_course_overlap)

# Rooms

In [11]:
R_df = pd.DataFrame(pd.read_csv('orie_rooms.csv', header=None))
R = R.to_numpy()
R

array([[253],
       [453],
       [571],
       [999]])

# Meeting Times

In [81]:
meeting_df = pd.DataFrame((pd.read_excel('4999_Enumeration.xlsx')))

def convert_time_to_minutes(time_str):
    hour, minute = time_str.split(':')
    return int(hour) * 60 + int(minute[:-2]) + (0 if time_str.endswith('am') else 12 * 60)
meeting_df['Start'] = meeting_df['Start'].apply(convert_time_to_minutes)
meeting_df['End'] = meeting_df['End'].apply(convert_time_to_minutes)
T = meeting_df

matrix_size = len(meeting_df)
overlap_matrix = np.zeros((matrix_size, matrix_size), dtype=int)
for i in range(matrix_size):
    for j in range(matrix_size):
        days_overlap = set(meeting_df.iloc[i]['Days'].split('/')).intersection(set(meeting_df.iloc[j]['Days'].split('/')))
        start_overlap = meeting_df.iloc[i]['Start'] <= meeting_df.iloc[j]['End'] and meeting_df.iloc[i]['End'] >= meeting_df.iloc[j]['Start']
        overlap_matrix[i, j] = 1 if days_overlap and start_overlap else 0

overlap_matrix_df = pd.DataFrame(overlap_matrix, index=meeting_df['Value'], columns=meeting_df['Value'].values)

# rename to match written IP
F = overlap_matrix_df
F

,1,2,3,4,5,6,7,8,9,10,...,27,28,29,30,31,32,33,34,35,36
Value,,,,,,,,,,,,,,,,,,,,,
1,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Last Year Schedule

In [85]:
# importing all of the sets into the code
# last year schedule file
last_year_schedule = pd.DataFrame(pd.read_csv('last_year_schedule_temp.csv'))
last_year_schedule



,course_id,room,time,y_crt
0,3150,253,1,1
1,3150,253,2,0
2,3150,253,3,0
3,3310,571,1,1
4,3310,571,2,0
5,3310,571,3,0
6,3510,453,1,0
7,3510,453,2,1
8,3510,453,3,0
9,3120,253,1,0


In [89]:
last_year_matrix = np.zeros((len(R), len(T), len(C)), dtype=int)

for _, row in last_year_schedule.iterrows():
    if row['y_crt'] == 1:
        course_index = np.where(C == row['course_id'])[0]
        time_index = np.where(T == row['time'])[0]
        room_index = np.where(R == row['room'])[0]

        # Ensure indices exist before assigning the value
        if course_index.size > 0 and time_index.size > 0 and room_index.size > 0:
            last_year_matrix[room_index[0], time_index[0], course_index[0]] = 1
last_year_matrix

array([[[0, 1, 0, 0],
        [0, 0, 0, 0],
        [1, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]],

       [[0, 0, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        